In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
%cd drive/MyDrive/Chatbot/

/content/drive/MyDrive/Chatbot


In [4]:
!ls

beagle.webp  images.jpeg       llama_vision.ipynb  requirements.txt  vision.ipynb
config.json  Llama_chat.ipynb  main.ipynb	   Untitled11.ipynb


In [5]:
!pip install torch transformers Pillow

In [6]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 62.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [7]:
from transformers import MllamaForConditionalGeneration, AutoProcessor, TextIteratorStreamer
from PIL import Image
import torch
from threading import Thread
import time

In [8]:
# Load the Llama model and processor
ckpt = "meta-llama/Llama-3.2-11B-Vision-Instruct"

In [9]:
from huggingface_hub import login
import json

# Load configuration
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
model = MllamaForConditionalGeneration.from_pretrained(ckpt, torch_dtype=torch.bfloat16, use_auth_token=HF_TOKEN)
processor = AutoProcessor.from_pretrained(ckpt, use_auth_token=HF_TOKEN)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3274: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/5.07k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/89.4k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

In [11]:
def generate_response(image_path, text_prompt, max_new_tokens=250):
    # Load the image
    image = Image.open(image_path).convert("RGB")

    # Prepare messages
    messages = [
        {"role": "user", "content": [{"type": "text", "text": text_prompt}, {"type": "image"}]},
    ]

    # Apply chat template
    texts = processor.apply_chat_template(messages, add_generation_prompt=True)

    # Prepare inputs for the model
    inputs = processor(text=texts, images=[image], return_tensors="pt")
    streamer = TextIteratorStreamer(processor, skip_special_tokens=True, skip_prompt=True)

    # Generate the response
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=max_new_tokens)
    generated_text = ""

    # Start generating in a separate thread
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    buffer = ""

    # Collect generated text
    for new_text in streamer:
        buffer += new_text
        generated_text = buffer
        time.sleep(0.01)

    # Wait for the thread to finish
    thread.join()

    return generated_text

In [12]:
# Example usage
if __name__ == "__main__":
    # Path to the image file
    image_path = "/content/drive/MyDrive/Chatbot/images.jpeg"
    text_prompt = "What can you tell me about this image?"

    response = generate_response(image_path, text_prompt)
    print("Response:", response)

KeyboardInterrupt: 